In [ ]:
# %%

from collections import Counter

import pyro
import torch
from tqdm import tqdm

d:\SAIT\aDATA475\VS_Python\lab7p\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# %%

def coin_toss(n):
    faces = Counter()
    total = 0
    for _ in tqdm(range(n)):

        # create a sample of Bernoulli distribution for fair (50/50) coin
        # The size of the sample is 1 by default for pyro.sample
        outcome = pyro.sample("coin", pyro.distributions.Bernoulli(0.5))

        # convert the Bernoulli distribution into meaning
        # what does 1/0 stands for ?
        if outcome.item() == 1:
            result = "head"
        else:
            result = "tail"

        # gain 2 if head is tossed, otherwise lose 2
        reward = {"head": 2, "tail": -2}[result]

        # update the faces Counter
        faces[result] += 1
        total += reward

    return faces, total


def coin_toss_tensor(n):

    # create a sample of Bernoulli distribution for fair (50/50) coin
    # The size of the sample is n
    outcome = pyro.sample("coin", pyro.distributions.Bernoulli(0.5).expand([n]))

    # return Counter object to summarize the counts of head/tail, and total rewards
    heads = torch.sum(outcome).item()
    tails = n - heads
    total_reward = 2 * heads - 2 * tails
    return Counter({"head": heads, "tail": tails}), total_reward


def simulation(n, simulation_func):
    faces, total = simulation_func(n)

    print(f"\nRan {n} simulation{'s' if n >1 else ''}")
    print(f"Total Reward = {total}")
    print(faces)

In [ ]:
# %%

for n in [1, 1000, 1000000]:
    simulation(n, coin_toss)

100%|██████████| 1/1 [00:00<00:00,  1.20it/s]



Ran 1 simulation
Total Reward = 2
Counter({'head': 1})


100%|██████████| 1000/1000 [00:00<00:00, 6540.59it/s]



Ran 1000 simulations
Total Reward = -32
Counter({'tail': 508, 'head': 492})


100%|██████████| 1000000/1000000 [01:44<00:00, 9527.81it/s]


Ran 1000000 simulations
Total Reward = 1868
Counter({'head': 500467, 'tail': 499533})


In [ ]:
# %%

for n in [1, 1000, 1000000]:
    simulation(n, coin_toss_tensor)


Ran 1 simulation
Total Reward = 2.0
Counter({'head': 1.0, 'tail': 0.0})

Ran 1000 simulations
Total Reward = 28.0
Counter({'head': 507.0, 'tail': 493.0})

Ran 1000000 simulations
Total Reward = 1128.0
Counter({'head': 500282.0, 'tail': 499718.0})
